In [27]:
from abenc_ph_mj18 import PH_ABE, mat_math, Inner_Product
from charm.toolbox.pairinggroup import PairingGroup, ZR, G1, G2, GT, pair
from charm.toolbox.ABEnc import ABEnc

In [28]:
def ad_auth_setup(n, assump_size, ad, pks):

    pk_ad = []
    k = assump_size
    pk_ad = [[1] * k] * (k+1)

    for i in range(n-1):
        if i + 1 == ad:
            continue

        g_XTA = pks[str(i+1)]['g_1^{X^T A}']

        tmp = []
        for j in range(len(g_XTA)):
            tmp.append([x / y for x,y in zip(pk_ad[j], g_XTA[j])])
        
        pk_ad = tmp

    pk = pks[str(ad)]
    pk['g_1^{X^T A}'] = pk_ad
    
    pks[str(ad)] = pk

    return pks
    

In [29]:
def gen_omega(K, C, n, assump_size):
    H = K['g_2^h']
    k = assump_size
    C_i_s = C['C_i']

    C_ = [1] * (k + 1)
    #C_  = tmp
    for i in range(n-1):
        #print (len(C_), len(C_i_s[i]))
        C_ = [x / y for x,y in zip(C_, C_i_s[i])]

        #print (len(C_))

    return [pair(x,y) for x,y in zip(C_, H)]

In [30]:
n = 10
assump_size = 3
group = PairingGroup('MNT224')
math_lib = mat_math()

ph_abe = PH_ABE(n, assump_size, group, math_lib)

pp, msk = ph_abe.setup()
attributes = Inner_Product(group)
vec_x, vec_v = attributes.gen_x_v(n, assump_size)

pks, sks = ph_abe.auth_setup(pp)

In [31]:
ad = vec_v.index(0) + 1
ad_vec_v = [0] * (n-1) + [1]
print ('Authorized list: ', vec_v)
print ('Adversary:', ad, '\nVector belongs to AD:', ad_vec_v)
pks = ad_auth_setup(n, assump_size, ad, pks)

Authorized list:  [0, 1, 1, 1, 1, 1, 0, 0, 1, 1]
Adversary: 1 
Vector belongs to AD: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [32]:
M = group.random(GT)
print ('M: ', M)

C, vec_s = ph_abe.encrypt(pp, pks, vec_x, M)

M:  [[5038973744866951840776065006013138420695322697979728145521730329146, 14837491594312303209833875621790332361050041132194652945679373446975, 8387522200116333543355963480288421764865029732676042571931533013585], [7123377325527833739013609321237147015736754457640859252072261192363, 14030993579633381150436508320825388690284692191115310974961482046524, 9271122366359626461875710011807701721513268385523080964172501148983]]


In [33]:
K = ph_abe.keygen(pp, sks, vec_v)
print ('M_ (Usr):', ph_abe.decrypt(K, C, vec_v,pp))

TypeError: PH_ABE._gen_mus() missing 2 required positional arguments: 'GID' and 'vec_v'

In [34]:
K_ = ph_abe.keygen(pp, sks, ad_vec_v)
omega = gen_omega(K_, C, n, assump_size)
M_ = ph_abe.decrypt(K_, C, ad_vec_v, pp)
print ('M_ (Adv):', M_ * math_lib.prod(omega))

TypeError: PH_ABE._gen_mus() missing 2 required positional arguments: 'GID' and 'vec_v'

In [67]:
class RPs:
    def __init__(self, s, G, group):
        self.s = s
        self.B = G ** s
        self.A = G
        self.group = group

    def getA(self):
        return self.A

    

In [85]:
class NIZK():

    def __init__(self, rps, group):
        self.rps = rps
        self.group = group

    def commit(self, h):
        a = self.group.random(ZR)
        R = self.rps.A ** a
        print (a, R)
        hash_input = str(R) + h
        c = self.group.hash(hash_input, ZR)
        print ("add 1:", c * self.rps.s)
        print ("add 2:", a)
        u = a + c * self.rps.s
        return R, u
    
    def verify(self, R, u, h):
            hash_input = str(R) + h
            c = self.group.hash(hash_input, ZR)

            return self.rps.A ** u == R * (self.rps.B ** c)
        
        



In [86]:
group = PairingGroup('MNT224')
s = group.random(ZR)

In [87]:
rps = RPs(s, group.random(G1), group)

In [88]:
nizk = NIZK(rps, group)

In [89]:
R, u = nizk.commit("abd")

2042505281892514900967974934741364031288178404995691350652557646367 [3697327398997224313162851902352986109970951457283345030364976871137, 7249121661250782700566430785868938224711204716581130614891784509962]
add 1: 5321640237532103160627954028918825727879448742358636889446439420332
add 2: 2042505281892514900967974934741364031288178404995691350652557646367


In [90]:
from charm.toolbox.pairinggroup import PairingGroup, ZR, G1, G2, GT, pair

In [92]:
nizk.verify(R, u, "abc")

False

In [78]:
G2

2